In [1]:
import sqlite3 as dbms

conn = dbms.connect("students.sqlite3")  # Как оригинально!
cursor = conn.cursor()

In [2]:
with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f:
    drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f:
    create_ddl = f.read()

In [3]:
if True:
    cursor.executescript(drop_ddl)
    conn.commit()
if True:
    cursor.executescript(create_ddl)
    conn.commit()

In [4]:
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

engine = sqlalchemy.create_engine('sqlite:///students.sqlite3', echo=False)  # echo=True для логгинга
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
null=True
class Program(DeclBase):
    __tablename__ = 'programs'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    programs_courses = relationship("ProgramCourseLink", backref="program")

    def __init__(self, name):
        self.name = name

class Student(DeclBase):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    marks = relationship("Mark", backref="student")
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program
        
class Course(DeclBase):
    __tablename__ = 'courses'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    programs_courses = relationship("ProgramCourseLink", backref="course")
    marks = relationship("Mark", backref="course")
    
    def __init__(self, name):
        self.name = name
        
class ProgramCourseLink(DeclBase):
    __tablename__ = 'programs_courses'
    semester_number = Column(Integer, primary_key=True)
    
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    program_id = Column(Integer, ForeignKey('programs.id'), primary_key=True)
    
    def __init__(self, semester_number, course, program):
        self.semester_number = semester_number
        self.course = course
        self.program = program
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    mark = Column(Integer)
    
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    
    def __init__(self, mark, student, course):
        self.mark = mark
        self.student = student
        self.course = course

In [7]:
first_program = Program("Математическая логика, алгебра и теория чисел")
second_program = Program("Теоретические основы информатики")

first_course = Course("Линейная алгебра")
second_course = Course("Теория чисел")
third_course = Course("Алгоритмы и структуры данных")

first_program_course_link = ProgramCourseLink(semester_number="1",
                                              course=first_course,
                                              program=first_program)
second_program_course_link = ProgramCourseLink(semester_number="3",
                                               course=second_course,
                                               program=first_program)
third_program_course_link = ProgramCourseLink(semester_number="2",
                                              course=third_course,
                                              program=second_program)

first_student = Student(card="0000001",
                        surname="Дементьев",
                        name="Святослав",
                        patronymic="Фёдорович",
                        program=first_program)
second_student = Student(card="000002",
                         surname="Харламов",
                         name="Владислав",
                         patronymic="Сергеевич",
                         program=first_program)
third_student = Student(card="000003",
                        surname="Тюменцев",
                        name="Максим",
                        patronymic="Михайлович",
                        program=first_program)
fourth_student = Student(card="000004",
                         surname="Щепелев",
                         name="Артур",
                         patronymic="Вячеславович",
                         program=second_program)
fifth_student = Student(card="000005",
                        surname="Путинцев",
                        name="Степан",
                        patronymic="Юрьевич",
                        program=second_program)

first_student_first_course_mark = Mark(mark="4", student=first_student, course=first_course)
first_student_second_course_mark = Mark(mark="4", student=first_student, course=second_course)
second_student_first_course_mark = Mark(mark="3", student=second_student, course=first_course)
second_student_second_course_mark = Mark(mark="3", student=second_student, course=second_course)
third_student_mark_first_course_mark = Mark(mark="5", student=third_student, course=first_course)
third_student_second_course_mark = Mark(mark="5", student=third_student, course=second_course)
fourth_student_third_course_mark = Mark(mark="4", student=fourth_student, course=third_course)
fifth_student_third_course_mark = Mark(mark="3", student=fifth_student, course=third_course)

session.add_all([
    first_student,
    second_student,
    third_student,
    fourth_student,
    fifth_student,

    first_course,
    second_course,
    third_course,

    first_student_first_course_mark,
    first_student_second_course_mark,
    second_student_first_course_mark,
    second_student_second_course_mark,
    third_student_mark_first_course_mark,
    third_student_second_course_mark,
    fourth_student_third_course_mark,
    fifth_student_third_course_mark,

    first_program_course_link,
    second_program_course_link,
    third_program_course_link
])
session.commit()

In [15]:
import sys

print("Программы и студенты:")
for program in session.query(Program):
    print("Программа:", program.name, end=" ")
    for s in program.programs_courses:
        print("(", "Номер семестра: ", s.semester_number, ")", sep="")
    for s in program.students:
        print("-",s.surname, s.name[0] + ".", s.patronymic[0] + ".")
    
print("\nСтуденты и их оценки:")
for student in session.query(Student):
    print(student.surname, ":", sep="",)
    for mark in student.marks:
        print("-", mark.course.name, mark.mark)

Программы и студенты:
Программа: Математическая логика, алгебра и теория чисел (Номер семестра: 1)
(Номер семестра: 3)
- Дементьев С. Ф.
- Харламов В. С.
- Тюменцев М. М.
Программа: Теоретические основы информатики (Номер семестра: 2)
- Щепелев А. В.
- Путинцев С. Ю.

Студенты и их оценки:
Дементьев:
- Линейная алгебра 4
- Теория чисел 4
Харламов:
- Линейная алгебра 3
- Теория чисел 3
Тюменцев:
- Теория чисел 5
- Линейная алгебра 5
Щепелев:
- Алгоритмы и структуры данных 4
Путинцев:
- Алгоритмы и структуры данных 3
